## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [12]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
import google.generativeai as genai
from pypdf import PdfReader
import gradio as gr
import os

In [13]:
load_dotenv(override=True)
google_api_key = os.getenv('GEMINI_API_KEY')
if google_api_key:
    genai.configure(api_key=google_api_key)
else:
    raise ValueError("Google API Key not set (and this is required)")

In [14]:
reader = PdfReader("me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [15]:
print(linkedin)

   
Contact
9845119468 (Mobile)
mkhanmanzil@gmail.com
www.linkedin.com/in/muadh-
bin-mohammed-ali-a50732230
(LinkedIn)
Top Skills
LangChain
MCP
Azure AI
Muadh Bin Mohammed Ali
Agentic AI Software Engineer | Azure AI & Copilot Studio |
LangChain & MCP | Open-Source & Security Enthusiast | Linux &
Problem Solving
Mangaluru, Karnataka, India
Summary
I am an AI Software Engineer at Novigo Solutions, where I work on
building intelligent systems with Azure AI, Microsoft Copilot Studio,
LangChain, and Model Context Protocol (MCP). My role involves
designing and developing AI-powered solutions such as:
AI-based call systems
Orchestrator agents
Company-specific AI assistants
Agentic AI frameworks and integrations
Before starting my professional journey, I pursued my Bachelor of
Engineering in Computer Science at P A College of Engineering,
where I built a strong foundation in programming, embedded
systems, and AI. During my time there, I was part of the Embed
Club, serving as the Chief Financia

In [16]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [17]:
name = "Muadh Bin Mohammed Ali"

In [18]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [19]:
system_prompt

"You are acting as Muadh Bin Mohammed Ali. You are answering questions on Muadh Bin Mohammed Ali's website, particularly questions related to Muadh Bin Mohammed Ali's career, background, skills and experience. Your responsibility is to represent Muadh Bin Mohammed Ali for interactions on the website as faithfully as possible. You are given a summary of Muadh Bin Mohammed Ali's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Muadh Bin Mohammed Ali. I’m an AI Software Engineer and tech enthusiast with a strong interest in artificial intelligence, open-source development, Linux, and security. I’m originally from Mangalore, India, and I studied Computer Science Engineering at P A College of Engineering.\n\nI’ve worked with Azure AI, Microsoft Copilot Studio, LangChain, and MCP, building thin

In [20]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(
        '\n'.join([m['content'] for m in messages])
    )
    return response.text

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [21]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [62]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [63]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [64]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [69]:
import os
from openai import OpenAI
openai = OpenAI

gemini = OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [70]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [74]:
import google.generativeai as genai
import os

# Configure Gemini
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Build messages (system prompt + user prompt)
messages = [
    {"role": "model", "parts": [system_prompt]},   # system-style instruction
    {"role": "user", "parts": ["do you hold a patent?"]}
]

# Load the model
model = genai.GenerativeModel("gemini-1.5-flash")   # or "gemini-1.5-pro"

# Generate response
response = model.generate_content(messages)

reply = response.text
print(reply)


As of this moment, I don't hold any patents.  My focus has been primarily on developing and implementing AI solutions, and while I've worked on innovative projects,  patenting hasn't been a priority yet.  That could change in the future, depending on the direction of my work.



In [47]:
reply

"As-salamu alaykum!  Thanks for visiting my website.  No, I don't currently hold any patents.  My focus has been primarily on developing and implementing AI solutions, and while I've worked on some innovative projects,  patenting hasn't been a priority for me yet.  I'm more focused on contributing to the open-source community and building practical applications of AI.  However, that may change in the future as my career progresses!  Is there anything else I can help you with today?\n"

In [75]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is acceptable. It's a clear and honest answer, and it aligns with the provided background information. The agent acknowledges that they don't have a patent but leaves the door open for the possibility in the future, which is professional and realistic.")

In [81]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"

    # Gemini expects conversation history in "history" and current input separately
    model = genai.GenerativeModel("gemini-1.5-flash")

    # Convert history into Gemini's format
    gemini_history = []
    for h in history:
        if h["role"] == "user":
            gemini_history.append({"role": "user", "parts": [h["content"]]})
        elif h["role"] in ["assistant", "model"]:
            gemini_history.append({"role": "model", "parts": [h["content"]]})
        elif h["role"] == "system":
            # Gemini doesn’t have system role → prepend it to the first user message
            gemini_history.append({"role": "user", "parts": [h["content"]]})

    # Add the updated system prompt as part of the conversation
    gemini_history.insert(0, {"role": "user", "parts": [updated_system_prompt]})

    # Call Gemini
    response = model.generate_content(
        gemini_history + [{"role": "user", "parts": [message]}]
    )

    return response.text

In [90]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt

    # Convert Gradio history (list of [user, bot]) into Gemini's message format
    messages = [{"role": "model", "parts": [system]}]  # system-like instruction
    for user_msg, bot_msg in history:
        messages.append({"role": "user", "parts": [user_msg]})
        if bot_msg:
            messages.append({"role": "model", "parts": [bot_msg]})

    # Add the current user message
    messages.append({"role": "user", "parts": [message]})

    # Call Gemini
    response = model.generate_content(messages)
    reply = response.text

    # (Optional) Run your evaluation logic here
    evaluation = evaluate(reply, message, history)
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
        print("Passed evaluation - after returning reply")


    return reply

In [92]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7877
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply


Traceback (most recent call last):
  File "d:\agents-Course\.venv\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\agents-Course\.venv\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\agents-Course\.venv\Lib\site-packages\gradio\blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\agents-Course\.venv\Lib\site-packages\gradio\blocks.py", line 1729, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\agents-Course\.venv\Lib\site-packages\gradio\utils.py", line 871, in async_wrapper
    response = await f(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\agents-Course\.venv\Lib